Adapted from https://github.com/catalystneuro/movshon-lab-to-nwb/blob/main/tutorials/blackrock_nwb_conversion_detailed.ipynb

In [2]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
!ls ../nwb-conversion-tools/nwb_conversion_tools -l

total 76
-rwxrwxrwx 1 steburg steburg    69 Apr  5 11:28 __init__.py
-rwxrwxrwx 1 steburg steburg  1404 Apr  5 11:28 auto_qc.py
-rwxrwxrwx 1 steburg steburg  1338 Apr  5 11:28 basedatainterface.py
-rwxrwxrwx 1 steburg steburg  2134 Apr  5 11:28 baseimagingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  2775 Apr  5 11:28 baselfpextractorinterface.py
-rwxrwxrwx 1 steburg steburg  4629 Apr  5 11:28 baserecordingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  2363 Apr  5 11:28 basesegmentationextractorinterface.py
-rwxrwxrwx 1 steburg steburg  3829 Apr  5 11:28 basesortingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  4312 Apr  5 11:28 conversion_tools.py
drwxrwxrwx 1 steburg steburg  4096 Apr  5 11:28 datainterfaces
-rwxrwxrwx 1 steburg steburg  5100 Apr  5 11:28 json_schema_utils.py
-rwxrwxrwx 1 steburg steburg 10154 Apr  5 11:28 metafile.schema.json
-rwxrwxrwx 1 steburg steburg  6665 Apr  5 11:28 nwbconverter.py
drwxrwxrwx 1 steburg steburg  4096 Apr  5 11:28 schemas
-rwxrw

In [18]:
# Import modules

import random
import string
import pytz
from typing import Union, Optional
from pathlib import Path
import spikeextractors as se
from pynwb import NWBFile

from nwb_conversion_tools.baserecordingextractorinterface import BaseRecordingExtractorInterface
from nwb_conversion_tools.basesortingextractorinterface import BaseSortingExtractorInterface
from nwb_conversion_tools.json_schema_utils import get_schema_from_method_signature
from nwb_conversion_tools.datainterfaces.interface_utils.brpylib import NsxFile

In [21]:
PathType = Union[str, Path, None]
print(PathType)

typing.Union[str, pathlib.Path, NoneType]


In [27]:
get_schema_from_method_signature?

In [22]:
RX = se.AxonaRecordingExtractor

In [23]:
RX

spikeextractors.extractors.neoextractors.axonaextractor.AxonaRecordingExtractor

In [41]:
import inspect

In [42]:
inspect.signature(RX.__init__).parameters

mappingproxy({'self': <Parameter "self">,
              'block_index': <Parameter "block_index=None">,
              'seg_index': <Parameter "seg_index=None">,
              'kargs': <Parameter "**kargs">})

In [48]:
keys = inspect.signature(RX.__init__).parameters.keys()
print(keys)

odict_keys(['self', 'block_index', 'seg_index', 'kargs'])


In [50]:
inspect.signature(RX.__init__).parameters['block_index'].annotation

inspect._empty

In [43]:
get_schema_from_method_signature(
    class_method=RX.__init__,
    exclude=None
)

ValueError: No valid arguments were found in the json type mapping!

In [40]:
metadata_schema = get_schema_from_method_signature(
        class_method=RX.__init__,
        exclude=['block_index', 'seg_index']
    )

ValueError: No valid arguments were found in the json type mapping!

In [34]:
RX.

spikeextractors.extractors.neoextractors.axonaextractor.AxonaRecordingExtractor

In [68]:
class AxonaRecordingExtractorInterface(BaseRecordingExtractorInterface):
    """Primary data interface class for converting a AxonaRecordingExtractor."""

    RX = se.AxonaRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        """Compile input schema for the RecordingExtractor."""
        metadata_schema = get_schema_from_method_signature(
            class_method=cls.RX.__init__,
            exclude=['block_index', 'seg_index']
        )
        metadata_schema['additionalProperties'] = True
        metadata_schema['properties']['filename']['format'] = 'file'
        metadata_schema['properties']['filename']['description'] = 'Path to Axona files.'
        return metadata_schema
    
    def __init__(self, filename: PathType, nsx_to_load: Optional[int] = None):
        super().__init__(filename=filename, nsx_to_load=nsx_to_load)

In [69]:
RXI = AxonaRecordingExtractorInterface(filename='axona_raw_5s')

TypeError: __init__() got an unexpected keyword argument 'nsx_to_load'

In [58]:
RXI.get_source_schema()

ValueError: No valid arguments were found in the json type mapping!

In [ ]:
"""Authors: Luiz Tauffer"""
import random
import string
import pytz
from typing import Union, Optional
from pathlib import Path
import spikeextractors as se
from pynwb import NWBFile

from nwb_conversion_tools.baserecordingextractorinterface import BaseRecordingExtractorInterface
from nwb_conversion_tools.basesortingextractorinterface import BaseSortingExtractorInterface
from ..json_schema_utils import get_schema_from_method_signature
from .interface_utils.brpylib import NsxFile

PathType = Union[str, Path, None]


class BlackrockRecordingExtractorInterface(BaseRecordingExtractorInterface):
    """Primary data interface class for converting a BlackrockRecordingExtractor."""

    RX = se.BlackrockRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        """Compile input schema for the RecordingExtractor."""
        metadata_schema = get_schema_from_method_signature(
            class_method=cls.RX.__init__,
            exclude=['block_index', 'seg_index']
        )
        metadata_schema['additionalProperties'] = True
        metadata_schema['properties']['filename']['format'] = 'file'
        metadata_schema['properties']['filename']['description'] = 'Path to Blackrock file.'
        return metadata_schema
    
    def __init__(self, filename: PathType, nsx_to_load: Optional[int] = None):
        super().__init__(filename=filename, nsx_to_load=nsx_to_load)

    def get_metadata(self):
        """Auto-fill as much of the metadata as possible. Must comply with metadata schema."""
        metadata = super().get_metadata()

        # Open file and extract headers
        nsx_file = NsxFile(datafile=self.source_data['filename'])
        session_start_time = nsx_file.basic_header['TimeOrigin']
        session_start_time_tzaware = pytz.timezone('EST').localize(session_start_time)
        comment = nsx_file.basic_header['Comment']

        metadata['NWBFile'] = dict(
            session_start_time=session_start_time_tzaware,
            session_description=comment,
            identifier=''.join(random.choices(string.ascii_uppercase + string.digits, k=12))
        )

        metadata['Ecephys'] = dict(
            Device=[dict(
                name='Device_ecephys',
                description='no description'
            )],
            ElectrodeGroup=[],
        )

        if self.source_data['nsx_to_load'] != 6:
            metadata['Ecephys']['LFPElectricalSeries'] = dict()
        else:
            metadata['Ecephys']['ElectricalSeries'] = dict()

        return metadata

    def run_conversion(self, nwbfile: NWBFile, metadata: dict = None, use_times: bool = False, 
                       write_as_lfp: bool = False, save_path: PathType = None, overwrite: bool = False, 
                       stub_test: bool = False):
        """
        Primary function for converting recording extractor data to nwb.
        Parameters
        ----------
        nwbfile: NWBFile
            nwb file to which the recording information is to be added
        metadata: dict
            metadata info for constructing the nwb file (optional).
            Should be of the format
                metadata['Ecephys']['ElectricalSeries'] = {'name': my_name,
                                                           'description': my_description}
        use_times: bool
            If True, the timestamps are saved to the nwb file using recording.frame_to_time(). If False (default),
            the sampling rate is used.
        write_as_lfp: bool (optional, defaults to False)
            If True, writes the traces under a processing LFP module in the NWBFile instead of acquisition.
        save_path: PathType
            Required if an nwbfile is not passed. Must be the path to the nwbfile
            being appended, otherwise one is created and written.
        overwrite: bool
            If using save_path, whether or not to overwrite the NWBFile if it already exists.
        stub_test: bool, optional (default False)
            If True, will truncate the data to run the conversion faster and take up less memory.
        """
        if self.source_data['nsx_to_load'] != 6:
            write_as_lfp = True

        super().run_conversion(
            nwbfile=nwbfile, 
            metadata=metadata, 
            use_times=use_times, 
            write_as_lfp=write_as_lfp,
            save_path=save_path, 
            overwrite=overwrite,
            stub_test=stub_test
        )


class BlackrockSortingExtractorInterface(BaseSortingExtractorInterface):
    """Primary data interface class for converting Blackrock spiking data."""

    SX = se.BlackrockSortingExtractor

    @classmethod
    def get_source_schema(cls):
        """Compile input schema for the RecordingExtractor."""
        metadata_schema = get_schema_from_method_signature(
            class_method=cls.SX.__init__,
            exclude=['block_index', 'seg_index', 'nsx_to_load']
        )
        metadata_schema['additionalProperties'] = True
        metadata_schema['properties']['filename']['format'] = 'file'
        metadata_schema['properties']['filename']['description'] = 'Path to Blackrock file.'
        return metadata_schema

    def __init__(self, filename: PathType, nsx_to_load: Optional[int] = None):
        super().__init__(filename=filename, nsx_to_load=nsx_to_load)

# Movshon lab - Blackrock Converter
This tutorial follows the step-by-step guide for a [NWB Converter](https://github.com/catalystneuro/nwb-conversion-tools/blob/master/documentation/conversion_tools_structure.md#step-by-step-operations)

In [3]:
from hussaini_lab_to_nwb import HussainiNWBConverter
from pynwb import NWBFile, NWBHDF5IO
from nwbwidgets import nwb2widget
from pathlib import Path
import yaml
import pprint

## Step 1 - Converter.get_source_schema()

In [6]:
# Get source_schema
source_schema = HussainiNWBConverter.get_source_schema()
pprint.pprint(source_schema['properties'], width=120)

{'BlackrockRecordingExtractorInterface': {'additionalProperties': True,
                                          'properties': {'filename': {'description': 'Path to Blackrock file.',
                                                                      'format': 'file',
                                                                      'type': 'string'},
                                                         'nsx_to_load': {'type': 'number'}},
                                          'required': ['filename'],
                                          'type': 'object'},
 'BlackrockSortingExtractorInterface': {'additionalProperties': True,
                                        'properties': {'filename': {'description': 'Path to Blackrock file.',
                                                                    'format': 'file',
                                                                    'type': 'string'}},
                                        'required': ['filename'

## Step 2 - Get user-input source_data that complies to the returned full source_schema

In [4]:
# Source data
base_path = Path('/media/luiz/storage/taufferconsulting/client_ben/project_movshon/data/')
file_recording = str(base_path / 'XX_LE_textures_20191128_002.ns3')
file_sorting = str(base_path / 'XX_LE_textures_20191128_002.nev')

source_data = dict(
    BlackrockRecordingExtractorInterface=dict(
        filename=file_recording,
        nsx_to_load=3
    ),
    BlackrockSortingExtractorInterface=dict(
        filename=file_sorting,
        nsx_to_load=6
    )
)

pprint.pprint(source_data, width=120)

{'BlackrockRecordingExtractorInterface': {'filename': '/media/luiz/storage/taufferconsulting/client_ben/project_movshon/data/XX_LE_textures_20191128_002.ns3',
                                          'nsx_to_load': 3},
 'BlackrockSortingExtractorInterface': {'filename': '/media/luiz/storage/taufferconsulting/client_ben/project_movshon/data/XX_LE_textures_20191128_002.nev',
                                        'nsx_to_load': 6}}


## Step 3 - Instantiate Converter

In [5]:
# Initialize converter
converter = MovshonBlackrockNWBConverter(source_data=source_data)

print('Data interfaces for this converter:')
pprint.pprint(converter.data_interface_objects, width=120)

Data interfaces for this converter:
{'BlackrockRecordingExtractorInterface': <nwb_conversion_tools.datainterfaces.blackrockdatainterface.BlackrockRecordingExtractorInterface object at 0x7f962d62a410>,
 'BlackrockSortingExtractorInterface': <nwb_conversion_tools.datainterfaces.blackrockdatainterface.BlackrockSortingExtractorInterface object at 0x7f962c5bd8d0>}


## Step 4 - Converter.get_metadata_schema()

In [6]:
# Get metadata_schema
metadata_schema = converter.get_metadata_schema()
pprint.pprint(metadata_schema, width=120)

{'$id': 'metadata.schema.json',
 '$schema': 'http://json-schema.org/draft-07/schema#',
 'additionalProperties': False,
 'description': 'Schema for the metadata',
 'properties': {'Ecephys': {'Device': {'additionalProperties': False,
                                       'properties': {'description': {'description': 'Description of the device (e.g., '
                                                                                     'model, firmware version, '
                                                                                     'processing software version, '
                                                                                     'etc.)',
                                                                      'type': 'string'},
                                                      'manufacturer': {'description': 'the name of the manufacturer of '
                                                                                      'this device',
            

## Step 5 - Automatically fetches available metadata with Converter.get_metadata()

In [7]:
# Get metadata from source data
metadata = converter.get_metadata()
pprint.pprint(metadata, width=120)

{'Ecephys': {'Device': [{'description': 'no description', 'name': 'Device_ecephys'}],
             'ElectrodeGroup': [],
             'LFPElectricalSeries': {}},
 'NWBFile': {'identifier': 'JES40X1FWSHT',
             'session_description': '',
             'session_start_time': datetime.datetime(2019, 11, 28, 22, 35, 54, 729000, tzinfo=<StaticTzInfo 'EST'>)}}


## Step 6 - Get user-input metadata

In [8]:
metadata['NWBFile']['session_description'] = 'example conversion'
pprint.pprint(metadata, width=120)

{'Ecephys': {'Device': [{'description': 'no description', 'name': 'Device_ecephys'}],
             'ElectrodeGroup': [],
             'LFPElectricalSeries': {}},
 'NWBFile': {'identifier': 'JES40X1FWSHT',
             'session_description': 'example conversion',
             'session_start_time': datetime.datetime(2019, 11, 28, 22, 35, 54, 729000, tzinfo=<StaticTzInfo 'EST'>)}}


## Step 7 - Converter.get_conversion_options_schema()

In [9]:
# Not used for this converter

## Step 8 - Get user-input conversion options

In [10]:
# Not used for this converter

## Step 9 - Run conversion user filled metadata and conversion_options

In [11]:
output_file = 'out_example.nwb'

converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file, 
    save_to_file=True,
    conversion_options=None
)

/home/luiz/anaconda3/envs/env_klusta/lib/python3.7/site-packages/hdmf/common/table.py:415: UserWarning: An attribute 'name' already exists on DynamicTable 'electrodes' so this column cannot be accessed as an attribute, e.g., table.name; it can only be accessed using other methods, e.g., table['name'].
  warn(msg)


NWB file saved at out_example.nwb!


## Final 1 - Check NWB file

In [12]:
# load file
fname = 'out_example.nwb'
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

root pynwb.file.NWBFile at 0x140281502197008
Fields:
  devices: {
    Device <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  file_create_date: [datetime.datetime(2021, 3, 5, 18, 47, 48, 213380, tzinfo=tzoffset(None, 3600))]
  identifier: JES40X1FWSHT
  processing: {
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_description: example conversion
  session_start_time: 2019-11-28 22:35:54.729000-05:00
  timestamps_reference_time: 2019-11-28 22:35:54.729000-05:00
  units: units <class 'pynwb.misc.Units'>



## Final 2 - Check NWB file with widgets

In [13]:
io = NWBHDF5IO(fname, 'r')
nwbfile = io.read()
nwb2widget(nwbfile)